# Treinando Sentence-BERT com Triplet Loss

Neste notebook, vamos treinar um modelo **Sentence-BERT (sBERT)** para medir similaridade entre sentenças usando a **Triplet Loss**.

A Triplet Loss treina o modelo comparando três frases por vez:
- Anchor (A) — frase base
- Positive (P) — frase parecida com A
- Negative (N) — frase diferente de A

A ideia é que a **distância(A, P) < distância(A, N)**

Ou seja: o modelo aprende a **aproximar os embeddings** de A e P, e **afastar os embeddings** de A e N.

## 1. Instalação e importação das bibliotecas

In [1]:
import os
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

Se você tiver acesso a uma **GPU** local, irá aparecer o nome abaixo:

In [2]:
print("CUDA disponível:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma GPU detectada")

CUDA disponível: True
GPU: NVIDIA GeForce GTX 1650


## 2. Carregar o dataset de treino

O arquivo `data/train.csv` contém pares de sentenças e um valor de similaridade (`similarity_score`) entre 0 e 5.

In [3]:
df = pd.read_csv("../data/train_triplet.csv")
df.head()

,anchor,positive,negative
0,"Who is the author of the book , `` The Iron La...",the IRON LADY ; A Biography of Margaret Thatch...,One of Qintex 's most visible assets remains t...
1,"Who is the author of the book , `` The Iron La...",the IRON LADY ; A Biography of Margaret Thatch...,Thus when it sent the message that the Norther...
2,"Who is the author of the book , `` The Iron La...",the IRON LADY ; A Biography of Margaret Thatch...,Agricultural output and the number of related ...
3,"Who is the author of the book , `` The Iron La...",the IRON LADY ; A Biography of Margaret Thatch...,It was not until the submarine 's trim equaliz...
4,"Who is the author of the book , `` The Iron La...",the IRON LADY ; A Biography of Margaret Thatch...,"The pilot , Chief Warrant Officer Bobby Hall ,..."


## 3. Preparar os exemplos para treino

O Sentence-BERT espera exemplos no formato `InputExample`, contendo dois textos e uma label (score de similaridade).

In [4]:
train_examples = [
    InputExample(
        texts=[row['anchor'], row['positive'], row['negative']]
    )
    for _, row in df.iterrows()
]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)  # type: ignore
print(f"✅ Total de triplas de treino: {len(train_examples)}")

✅ Total de triplas de treino: 1740


## 4. Carregar o modelo sBERT

Usaremos o modelo `multi-qa-MiniLM-L6-cos-v1` é uma versão multilíngue e otimizada do Sentence-BERT,
treinada em múltiplas tarefas de Pergunta–Resposta (*Question Answering*) e busca semântica (*semantic search*).

Ele é capaz de gerar embeddings comparáveis entre frases em diferentes idiomas, incluindo o português.

Também ativa o treinamento pela GPU, se estiver disponível.

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 5. Definir a função de perda Triplet Loss

A função `Triplet Loss` tenta **diminuir a distância** entre o par positivo e **aumentar** entre o par negativo.

In [14]:
train_loss = losses.TripletLoss(
    model=model,
    distance_metric=losses.TripletDistanceMetric.COSINE,  # usa distância de cosseno
    triplet_margin=0.3  # margem mínima entre positivo e negativo
)

## 6. Treinar o modelo

In [16]:
output_dir = '../output/sbert-triplet'
os.makedirs(output_dir, exist_ok=True)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=8,               # número de épocas no treinamento
    warmup_steps=100,       # número de passos para aquecimento, evita overfitting
    output_path=output_dir, # diretório para salvar o modelo
    show_progress_bar=True, # mostra barra de progresso
)

print(f"✅ Treinamento concluído! Modelo salvo em: {output_dir}")

Step,Training Loss
500,0.000100


✅ Treinamento concluído! Modelo salvo em: ../output/sbert-triplet


## Conclusão

O modelo foi treinado com **Triplet Loss**, que ensina a distinguir exemplos semelhantes e diferentes a partir de uma âncora (`anchor`).

Essa abordagem é muito usada em **busca semântica**, **detecção de duplicatas** e **recomendação de respostas**.